In [2]:
!pip install pymongo pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 KB 966.3 kB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.


In [3]:
# 1. Menghubungkan ke Database MongoDB
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')

Connected to MongoDB


In [53]:
# 2. Operasi CRUD Dasar
# Contoh Insert Data
employee_data = {
    'name': 'Alice',
    'department': 'Finance',
    'age': 29,
    'salary': 4500
}
collection.insert_one(employee_data)
print('Data inserted')

Data inserted


In [54]:
# Tugas 2: Insert, Update, dan Delete data pada koleksi 'employees'
# Menambahkan 200 data random dengan gaji di atas dan di bawah rata-rata

from pymongo import MongoClient
import random

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']
print("Connected to MongoDB")

# Insert Data Awal
employee_data = {
    'name': 'Alice',
    'department': 'Finance',
    'age': 29,
    'salary': 4500
}
collection.insert_one(employee_data)
print("Data awal berhasil dimasukkan!")

# Insert 200 Data Random
names = ['Rina', 'Budi', 'Andi', 'Siti', 'Tono', 'Alya', 'Rizky', 'Nanda', 'Lina', 'Eko']
departments = ['Finance', 'IT', 'HR', 'Marketing', 'Operations']
data_random = []

for i in range(200):
    name = random.choice(names) + f"_{i}"
    dept = random.choice(departments)
    age = random.randint(22, 50)
    # gaji acak di bawah dan di atas rata-rata
    if random.random() < 0.5:
        salary = random.randint(3000, 4500)
    else:
        salary = random.randint(4501, 7000)
    data_random.append({
        "name": name,
        "department": dept,
        "age": age,
        "salary": salary
    })

collection.insert_many(data_random)
print("200 data random berhasil dimasukkan ke koleksi 'employees'!")

# Query Data
print("\nContoh beberapa data dalam koleksi:")
for employee in collection.find().limit(10):
    print(employee)

# Update Data
collection.update_one(
    {"name": "Alice"},
    {"$set": {"salary": 6000}}
)
print("\nData karyawan 'Alice' berhasil diupdate!")

# Delete Data
collection.delete_one({"name": "Rina_5"})
print("Data karyawan 'Rina_5' berhasil dihapus.")

Connected to MongoDB
Data awal berhasil dimasukkan!
200 data random berhasil dimasukkan ke koleksi 'employees'!

Contoh beberapa data dalam koleksi:
{'_id': ObjectId('6902e603dee11fbf4c6e062b'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 6000}
{'_id': ObjectId('69037368823810ee22c0aae0'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('69037368823810ee22c0aae1'), 'name': 'Eko_0', 'department': 'IT', 'age': 34, 'salary': 3042}
{'_id': ObjectId('69037368823810ee22c0aae3'), 'name': 'Siti_2', 'department': 'Finance', 'age': 41, 'salary': 6253}
{'_id': ObjectId('69037368823810ee22c0aae4'), 'name': 'Rizky_3', 'department': 'Marketing', 'age': 48, 'salary': 5158}
{'_id': ObjectId('69037368823810ee22c0aae5'), 'name': 'Siti_4', 'department': 'Operations', 'age': 38, 'salary': 6114}
{'_id': ObjectId('69037368823810ee22c0aae6'), 'name': 'Rizky_5', 'department': 'IT', 'age': 50, 'salary': 3788}
{'_id': ObjectId('69037368823810ee22c0aae7'),

In [55]:
# Contoh Query Data
for employee in collection.find():
    print(employee)

{'_id': ObjectId('6902e603dee11fbf4c6e062b'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 6000}
{'_id': ObjectId('69037368823810ee22c0aae0'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('69037368823810ee22c0aae1'), 'name': 'Eko_0', 'department': 'IT', 'age': 34, 'salary': 3042}
{'_id': ObjectId('69037368823810ee22c0aae3'), 'name': 'Siti_2', 'department': 'Finance', 'age': 41, 'salary': 6253}
{'_id': ObjectId('69037368823810ee22c0aae4'), 'name': 'Rizky_3', 'department': 'Marketing', 'age': 48, 'salary': 5158}
{'_id': ObjectId('69037368823810ee22c0aae5'), 'name': 'Siti_4', 'department': 'Operations', 'age': 38, 'salary': 6114}
{'_id': ObjectId('69037368823810ee22c0aae6'), 'name': 'Rizky_5', 'department': 'IT', 'age': 50, 'salary': 3788}
{'_id': ObjectId('69037368823810ee22c0aae7'), 'name': 'Andi_6', 'department': 'IT', 'age': 36, 'salary': 3900}
{'_id': ObjectId('69037368823810ee22c0aae8'), 'name': 'Rina_7', 'department': 'Mark

In [56]:
# 3. Query Lebih Kompleks Menggunakan Aggregation
# Tugas 3: Terapkan aggregation untuk menghitung rata-rata gaji per departemen
# Aggregation untuk menghitung rata-rata gaji per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]

# Menjalankan aggregation dan menampilkan hasilnya
print("Rata-rata Gaji per Departemen\n")
for result in collection.aggregate(pipeline):
    print(result)

Rata-rata Gaji per Departemen

{'_id': 'Finance', 'average_salary': 4742.429718875502}
{'_id': None, 'average_salary': None}
{'_id': 'Marketing', 'average_salary': 4781.748743718593}
{'_id': 'IT', 'average_salary': 4795.009661835748}
{'_id': 'Operations', 'average_salary': 4857.5307017543855}
{'_id': 'HR', 'average_salary': 4817.1278538812785}


In [60]:
# 4. LATIHAN TAMBAHAN
# Latihan 4.1, 4.2, dan 4.3 (100 data produk random)

# Latihan 4.1
# Buat koleksi baru 'products' dan masukkan 100 data produk random
product_names = ["Laptop", "Mouse", "Keyboard", "Monitor", "Smartphone", "Tablet",
                 "Headphone", "Charger", "Speaker", "Camera", "Notebook", "Pen", "Desk", "Chair", "Lamp"]
categories = ["Electronics", "Accessories", "Furniture", "Stationery"]

products_data = []
for i in range(100):
    name = random.choice(product_names) + f"_{i}"
    category = random.choice(categories)
    price = random.randint(50, 15000)  # harga acak dari 50 hingga 15.000
    products_data.append({
        "name": name,
        "category": category,
        "price": price
    })
# Hapus data lama agar tidak duplikat
collection.delete_many({})

collection.insert_many(products_data)
print("100 data produk berhasil dimasukkan ke koleksi 'products'.")

# Mengecek dengan menampilkan 10 data pertama
print("\n📋 Beberapa data produk yang tersimpan:")
for product in collection.find().limit(10):
    print(product)

100 data produk berhasil dimasukkan ke koleksi 'products'.

📋 Beberapa data produk yang tersimpan:
{'_id': ObjectId('69038cb5823810ee22c0b779'), 'name': 'Laptop_0', 'category': 'Accessories', 'price': 13313}
{'_id': ObjectId('69038cb5823810ee22c0b77a'), 'name': 'Mouse_1', 'category': 'Furniture', 'price': 4313}
{'_id': ObjectId('69038cb5823810ee22c0b77b'), 'name': 'Lamp_2', 'category': 'Stationery', 'price': 8003}
{'_id': ObjectId('69038cb5823810ee22c0b77c'), 'name': 'Desk_3', 'category': 'Stationery', 'price': 7188}
{'_id': ObjectId('69038cb5823810ee22c0b77d'), 'name': 'Smartphone_4', 'category': 'Accessories', 'price': 10817}
{'_id': ObjectId('69038cb5823810ee22c0b77e'), 'name': 'Monitor_5', 'category': 'Accessories', 'price': 10852}
{'_id': ObjectId('69038cb5823810ee22c0b77f'), 'name': 'Lamp_6', 'category': 'Electronics', 'price': 10533}
{'_id': ObjectId('69038cb5823810ee22c0b780'), 'name': 'Tablet_7', 'category': 'Stationery', 'price': 13976}
{'_id': ObjectId('69038cb5823810ee22c0b

In [61]:
# Latihan 4.2 
# Query untuk menemukan produk dengan harga di atas rata-rata
# Hitung harga rata-rata
prices = [p["price"] for p in collection.find() if "price" in p]
average_price = statistics.mean(prices)

print(f"Produk dengan harga di atas rata-rata ({average_price:.2f})\n")
for product in collection.find({"price": {"$gt": average_price}}):
    print(product)

Produk dengan harga di atas rata-rata (6943.39)

{'_id': ObjectId('69038cb5823810ee22c0b779'), 'name': 'Laptop_0', 'category': 'Accessories', 'price': 13313}
{'_id': ObjectId('69038cb5823810ee22c0b77b'), 'name': 'Lamp_2', 'category': 'Stationery', 'price': 8003}
{'_id': ObjectId('69038cb5823810ee22c0b77c'), 'name': 'Desk_3', 'category': 'Stationery', 'price': 7188}
{'_id': ObjectId('69038cb5823810ee22c0b77d'), 'name': 'Smartphone_4', 'category': 'Accessories', 'price': 10817}
{'_id': ObjectId('69038cb5823810ee22c0b77e'), 'name': 'Monitor_5', 'category': 'Accessories', 'price': 10852}
{'_id': ObjectId('69038cb5823810ee22c0b77f'), 'name': 'Lamp_6', 'category': 'Electronics', 'price': 10533}
{'_id': ObjectId('69038cb5823810ee22c0b780'), 'name': 'Tablet_7', 'category': 'Stationery', 'price': 13976}
{'_id': ObjectId('69038cb5823810ee22c0b783'), 'name': 'Tablet_10', 'category': 'Electronics', 'price': 14027}
{'_id': ObjectId('69038cb5823810ee22c0b784'), 'name': 'Keyboard_11', 'category': 'Ac

In [62]:
# Latihan 4.3
# Aggregation pipeline untuk menghitung total produk per kategori
pipeline = [
    {"$group": {"_id": "$category", "total_products": {"$sum": 1}}}
]

print("Total produk dalam setiap kategori:\n")
for result in collection.aggregate(pipeline):
    print(result)

Total produk dalam setiap kategori:

{'_id': 'Furniture', 'total_products': 27}
{'_id': 'Stationery', 'total_products': 29}
{'_id': 'Electronics', 'total_products': 19}
{'_id': 'Accessories', 'total_products': 25}


In [69]:
# 5. TUGAS

# TUGAS 1
# Cari 5 karyawan dengan gaji tertinggi dalam setiap departemen
pipeline_top5 = [
    {'$sort': {'department': 1, 'salary': -1}},  # Urutkan per departemen dan gaji menurun
    {'$group': {
        '_id': '$department',
        'top_employees': {
            '$push': {
                'name': '$name',
                'salary': {'$ifNull': ['$salary', 0]}  # kalau salary kosong, isi 0
            }
        }
    }},
    {'$project': {
        'top_5': {'$slice': ['$top_employees', 5]}  # Ambil hanya 5 teratas
    }}
]

print("Karyawan dengan Gaji Tertinggi per Departemen\n")
for result in collection.aggregate(pipeline_top5):
    print(f"Departemen: {result['_id']}")
    for emp in result['top_5']:
        name = emp.get('name', 'Tidak ada nama')
        salary = emp.get('salary', 0)
        print(f"  {name} - Gaji: {salary}")
    print()

Karyawan dengan Gaji Tertinggi per Departemen

Departemen: Finance
  Eko_151 - Gaji: 6779
  Alya_166 - Gaji: 6655
  Budi_119 - Gaji: 6611
  Budi_57 - Gaji: 6512
  Budi_102 - Gaji: 6264

Departemen: None
  Laptop_0 - Gaji: 0
  Mouse_1 - Gaji: 0
  Lamp_2 - Gaji: 0
  Desk_3 - Gaji: 0
  Smartphone_4 - Gaji: 0

Departemen: HR
  Rizky_82 - Gaji: 6979
  Nanda_55 - Gaji: 6901
  Andi_168 - Gaji: 6820
  Andi_105 - Gaji: 6796
  Rizky_120 - Gaji: 6538

Departemen: IT
  Rizky_35 - Gaji: 6986
  Siti_87 - Gaji: 6878
  Nanda_97 - Gaji: 6802
  Andi_61 - Gaji: 6746
  Tono_4 - Gaji: 6421

Departemen: Marketing
  Andi_34 - Gaji: 6813
  Nanda_111 - Gaji: 6745
  Nanda_30 - Gaji: 6665
  Nanda_98 - Gaji: 6355
  Siti_129 - Gaji: 6139

Departemen: Operations
  Alya_84 - Gaji: 6828
  Siti_28 - Gaji: 6710
  Rina_49 - Gaji: 6656
  Rina_75 - Gaji: 6654
  Lina_114 - Gaji: 6207

